In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import feature_column

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from tensorflow.keras.utils import plot_model

import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
DATA_PATH = 'C:\git\SoloLearnML\stackoverflow\TextDataset.csv'

#it is just two column csv, like:
# text;label
# A wiki is run using wiki software;0
# otherwise known as a wiki engine.;1

dataframe = pd.read_csv(DATA_PATH, delimiter = ';')
dataframe.head()

# Or use this dummy dataframe
# d = {'text': ['A wiki is run using wiki software',
#               'otherwise known as a wiki engine.',
#               'otherwise known as a wiki engine.',
#               'A wiki engine is a type of content management system,',
#               'but it differs from most other such systems,including blog software',
#               'in that the content is created without any defined owner or leader',
#               'and wikis have little inherent structure',
#               'allowing structure to emerge according to the needs of the users',
#               'There are dozens of different wiki engines in use',
#               'both standalone and part of other software, such as bug tracking systems'
#              ], 
#      'label': [1,0,1,1,1,0,0,1,0,0]}
# dataframe = pd.DataFrame(data=d)

In [ ]:
# Preprocessing before feature_clolumn includes
# - getting the vocabulary
# - tokenization, which means only splitting on tokens. Encoding sentences with vocablary will be done by feature_column!
# - padding
# - truncating

# Build vacabulary
vocab_size = 100
oov_tok = '<OOV>'

sentences = dataframe['text'].to_list()

tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")

tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

# if word_index shorter then default value of vocab_size we'll save actual size
vocab_size=len(word_index)
print("vocab_size = word_index = ",len(word_index))

In [ ]:
# Split sentensec on tokens. here token = word
# text_to_word_sequence() has good default filter for charachters include basic punctuation, tabs, and newlines
dataframe['text'] = dataframe['text'].apply(tf.keras.preprocessing.text.text_to_word_sequence)

In [ ]:
dataframe.head()

In [ ]:
max_length = 6

# paddind and trancating setnences
# do that directly with strings without using tokenizer.texts_to_sequences()
# the feature_colunm will convert strings into numbers
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: (x + N * [''])[:N])
dataframe['text']=dataframe['text'].apply(lambda x, N=max_length: x[:N])
dataframe.head()

In [ ]:
# Define method to create tf.data dataset from Pandas Dataframe
def df_to_dataset(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    #labels = dataframe.pop(label_column)
    labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
# Split dataframe into train and validation sets
train_df, val_df = train_test_split(dataframe, test_size=0.2)

print(len(train_df), 'train examples')
print(len(val_df), 'validation examples')

In [ ]:
batch_size = 32
ds = df_to_dataset(dataframe, 'label',shuffle=False,batch_size=batch_size)

train_ds = df_to_dataset(train_df, 'label', batch_size=batch_size)
val_ds = df_to_dataset(val_df, 'label', shuffle=False, batch_size=batch_size)

In [ ]:
# and small batch for demo
example_batch = next(iter(ds))[0]
example_batch

In [ ]:
# Helper method to print exxample outputs of for defined feature_column

def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())
    
def seqdemo(feature_column):
    feature_layer = tf.keras.experimental.SequenceFeatures(feature_column)
    print(feature_layer(example_batch))

In [ ]:
# Define categorical colunm for our text feature, which is preprocessed into lists of tokens
# Note that key name should be the same as original column name in dataframe
text_column = feature_column.sequence_categorical_column_with_vocabulary_list(key='text', 
                                                                     vocabulary_list=list(word_index))

# arguemnt dimention here is exactly the dimension of the space in which tokens will be presented during model's learning
# see the tutorial at https://www.tensorflow.org/beta/tutorials/text/word_embeddings
text_embedding = feature_column.embedding_column(text_column, dimension=8)
print(seqdemo(text_embedding))

In [ ]:
# The define the layers and model it self
# This example uses Keras Functional API instead of Sequential just for more generallity

# Define SequenceFeatures layer to pass feature_columns into Keras model
sequence_feature_layer = tf.keras.experimental.SequenceFeatures(text_embedding)

In [ ]:
# Define inputs for each feature column. See
# см. https://github.com/tensorflow/tensorflow/issues/27416#issuecomment-502218673
feature_layer_inputs = {}
sequence_feature_layer_inputs = {}

# Here we have just one column

sequence_feature_layer_inputs['text'] = tf.keras.Input(shape=(max_length,), name='text', dtype=tf.string)
print(sequence_feature_layer_inputs)

In [ ]:
sequence_feature_layer(sequence_feature_layer_inputs)

In [ ]:
sequence_feature_layer_outputs, _ = sequence_feature_layer(sequence_feature_layer_inputs)
print(sequence_feature_layer_outputs)

In [ ]:
# Define outputs of SequenceFeatures layer 
# And accually use them as first layer of the model

# note here that SequenceFeatures layer produce tuple of two tensors as output. We need just first to pass next.
sequence_feature_layer_outputs, _ = sequence_feature_layer(sequence_feature_layer_inputs)
print(sequence_feature_layer_outputs)
# Add consequences layers. See https://keras.io/getting-started/functional-api-guide/

# Conv1D and MaxPooling1D will learn features from words order
x = tf.keras.layers.Conv1D(8,4)(sequence_feature_layer_outputs)
x = tf.keras.layers.MaxPooling1D(2)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x= tf.keras.layers.GlobalAveragePooling1D()(x)
# This example supposes binary classification, as labels are 0 or 1
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=[v for v in sequence_feature_layer_inputs.values()], outputs=x)

model.summary()

# This example supposes binary classification, as labels are 0 or 1
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              #run_eagerly=True
             )

In [ ]:
# Note that fit() method looking up features in train_ds and valdation_ds by name in 
# tf.keras.Input(shape=(max_length,), name='text'

# This model of cause will learn nothing because of fake data.

num_epochs = 5
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=num_epochs,
                    verbose=1
                    )